In [36]:
class CFPB:
    """This class helps you initialise perform operations on a Python class object called CFPB"""
    def __init__(self, base_url = None, name=None, creationTimeStamp=None,  modifiedTimeStamp=None, createdBy=None, modifiedBy=None, start_date=None, end_date=None, has_narrative=None, dataframes=[]) -> None:
        
        import datetime

        # Initialisation of attributes
        self.base_url = None
        self.name = None
        self.creationTimeStamp = None
        self.modifiedTimeStamp = None
        self.createdBy = None
        self.modifiedBy = None
        self.start_date = datetime.datetime.now()
        self.end_date = datetime.datetime.now()
        self.has_narrative = None
        self.dataframes = []

        # Assign attributes based on what's passed

        self.base_url = base_url if base_url else "https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/"
        self.name = name if name else self.name
        self.creationTimeStamp = creationTimeStamp if creationTimeStamp else datetime.datetime.now()
        self.modifiedTimeStamp = modifiedTimeStamp if modifiedTimeStamp else datetime.datetime.now()
        self.createdBy = createdBy if createdBy else self.createdBy
        self.modifiedBy = modifiedBy if modifiedBy else self.modifiedBy
        self.start_date = datetime.datetime.now() if start_date else self.start_date
        self.end_date = datetime.datetime.now() if end_date else self.end_date
        self.has_narrative = has_narrative if has_narrative else self.has_narrative
        self.dataframes = dataframes if dataframes else self.dataframes

    def create_request_url(self,base_url, start_date, end_date, has_narrative):
        """This function creates a request URL based on the parameters passed"""
        request_url = base_url + "?format=json&date_received_min=" + start_date + "&date_received_max=" + end_date + "&has_narrative=" + has_narrative
        return request_url

    def get_data(self,request_url):
        """This function gets data from the request URL"""
        import requests
        response = requests.get(request_url)
        if response.status_code == 200:
            return response.status_code, response.json()
        else:
            return response.status_code, {"error": "An error occurred while fetching data", "status_code": response.status_code, "reason": response.reason}
        
    def convert_data_to_dataframe(self,data):
        """This function converts data to a pandas dataframe"""
        import pandas as pd
        return pd.DataFrame(data)       
     
    def load(self, start_date=None, end_date=None, has_narrative=None):
        """This function loads data based on the parameters passed"""
        import datetime
        self.start_date = min(self.start_date, datetime.datetime.strptime(start_date, "%Y-%m-%d")) if start_date else self.start_date
        self.end_date = max(self.end_date, datetime.datetime.strptime(end_date, "%Y-%m-%d")) if end_date else self.end_date
        self.has_narrative = has_narrative if has_narrative else self.has_narrative
        request_url = self.create_request_url(self.base_url, start_date, end_date, has_narrative)
        status_code, data = self.get_data(request_url)
        print(status_code)
        if status_code == 200:
            dataframe = self.convert_data_to_dataframe([ob["_source"]for ob in data])
            self.dataframes.append(dataframe)
            print(f"Data frame of {len(data)} rows and {len(dataframe.columns)} columns loaded")
        else:
            print(data)

In [39]:
new_cfpb = CFPB()

In [43]:
new_cfpb.load("2024-01-01", "2024-01-31", "true")

200
Data frame of 43967 rows and 18 columns created


,product,complaint_what_happened,date_sent_to_company,issue,sub_product,zip_code,tags,complaint_id,timely,consumer_consent_provided,company_response,submitted_via,company,date_received,state,consumer_disputed,company_public_response,sub_issue
0,"Payday loan, title loan, personal loan, or adv...","I sent a communication to GOLDMAN SACHS, regar...",2024-01-13T12:00:00-05:00,Improper use of your report,Personal line of credit,XXXXX,None,8154407,Yes,Consent provided,Closed with explanation,Web,GOLDMAN SACHS BANK USA,2024-01-13T12:00:00-05:00,FL,N/A,None,Reporting company used your report improperly
1,Credit reporting or other personal consumer re...,"As a consumer, I am compelled to file this urg...",2024-01-14T12:00:00-05:00,Incorrect information on your report,Credit reporting,11520,None,8156277,Yes,Consent provided,Closed with non-monetary relief,Web,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",2024-01-13T12:00:00-05:00,NY,N/A,Company has responded to the consumer and the ...,Account information incorrect
2,Credit reporting or other personal consumer re...,I sent Transunion a letter on XXXX informing t...,2024-01-13T12:00:00-05:00,Improper use of your report,Credit reporting,764XX,Servicemember,8154520,Yes,Consent provided,Closed with non-monetary relief,Web,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",2024-01-13T12:00:00-05:00,TX,N/A,Company has responded to the consumer and the ...,Reporting company used your report improperly
3,Credit reporting or other personal consumer re...,"I opened up this account in XX/XX/2020, this w...",2024-01-14T12:00:00-05:00,Incorrect information on your report,Credit reporting,XXXXX,None,8156099,Yes,Consent provided,Closed with explanation,Web,Self Financial Inc.,2024-01-14T12:00:00-05:00,CA,N/A,None,Old information reappears or never goes away
4,Credit reporting or other personal consumer re...,The Following Accounts Were Opened Fraudulentl...,2024-01-22T12:00:00-05:00,Incorrect information on your report,Credit reporting,37013,None,8195318,Yes,Consent provided,Closed with non-monetary relief,Web,"EQUIFAX, INC.",2024-01-22T12:00:00-05:00,TN,N/A,None,Information belongs to someone else
